# Deep Reinforcement Learning Udacity Nanodegree
# Project 3: Collaboration and Competition

## Environment description
In the [Collaboration and Competition Project](https://github.com/udacity/deep-reinforcement-learning/tree/master/p3_collab-compet) in the [Deep Reinforcement Learning Udacity Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893), the Tennis environment has been slightly adapted by Udacity, so there might be differences to the original environment.

[//]: # (Image References)

[image1]: https://user-images.githubusercontent.com/10624937/42135623-e770e354-7d12-11e8-998d-29fc74429ca2.gif "Trained Agent"

![image1]

In this environment, two agents control rackets to bounce a ball over a net. If an agent hits the ball over the net, it receives a reward of +0.1.  If an agent lets a ball hit the ground or hits the ball out of bounds, it receives a reward of -0.01.  Thus, the goal of each agent is to keep the ball in play.

The observation space consists of 8 variables corresponding to the position and velocity of the ball and racket. Each agent receives its own, local observation.  Two continuous actions are available, corresponding to movement toward (or away from) the net, and jumping. 


In [3]:
env = UnityEnvironment(file_name='Tennis_Linux/Tennis.x86_64')

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		goal_size -> 5.0
		goal_speed -> 1.0
Unity brain name: ReacherBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 33
        Number of stacked Vector Observation: 1
        Vector Action space type: continuous
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


## Approach: Deep Deterministic Policy Gradient method

Due to the continuous action space, classical value-based methods are hard to apply. Instead, we solve the environment using the Multiagent Deep Deterministic Policy Gradient (DDPG) method. This is a flavour of an actor-critic method, where the actor is trained to maximize the expected outcome as is predicted by the critic network. As a twist, our critics also see the observation space and next actions of the opponents. This is allowed, as this only happens during training. When we test the agents, we don't need the critics anymore, and the actors only rely on their local observations.


### Implementation
I worked on this project as part of the [Deep Reinforcement Learning Udacity Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893) and turned the DDPG agent code from the [Pendulum project in Udacity's Deep Reinforcement Learning Repository](https://github.com/udacity/deep-reinforcement-learning/tree/master/ddpg-pendulum) into a Multiagent version. The most important changes are as follows.

#### Multiagent version
The original code only has a DDPG agent with actor and critic. In our environment, we have two agents each with their own actor and critic and own observations.

Computing the next actions is pretty straightforward, as the actor only needs to know about the current state.

However, during learning, we also need to compute and improve the critic, which is more complicated as it not only relies on the  observations and actions of the current agent, but also the opponent's observations and actions. Please see the publication (Multi Agent Actor Critic for Mixed Cooperative Competitive environments)[https://papers.nips.cc/paper/7217-multi-agent-actor-critic-for-mixed-cooperative-competitive-environments.pdf] by OpenAI for details.

From a technical perspective, this means that when computing the Q targets
```
Q_targets = r + γ * critic_target(next_state, actor_target(next_state))
```
we also need to incorporate the actor_target of the other actor. The same applies when we want to compute the current actor's loss, because then we also need the critic that takes the other actor's action into account.

In order to shield the Jupyter Notebook user from these details, I implemented the class ```

#### Ornstein-Uhlenbeck noise

In the original code, the noise is generated using this code here

```
dx = self.theta * (self.mu - x) + self.sigma * np.array([random.random() for i in range(len(x))])
```
However, `random.random()` generates uniformly distributed random numbers with mean 0.5. I felt I should change it to standard normally distributed values as is customary for Ornstein-Uhlenbeck by replacing `random.random()` by `np.random.randn()`. I also reduced the noise level by reducing the `sigma` parameter from `0.2` to `0.1`.

#### Shared experience buffer
There are at least two ways how multiple agents can learn from each other: By sharing
- experiences or
- network weights.

While the latter is the case for popular actor-critic algorithms like A2C or A3C, I decided to first try sharing the experience replay buffer. To that end, it is instantiated once and then passed to the constructors of the agents when instantiating the agents. Please note that we don't have true computational parallelism when training, so race conditions/thread safeness is not an issue here.

#### Gradient clipping in the critic training
In the course materials, it was mentioned that the training of the critic could benefit from gradient clipping, so I introduced the following line of code before performing `self.critic_optimizer.step()`:
```
torch.nn.utils.clip_grad_norm_(self.critic_local.parameters(), 1)
```

#### Deep Neural Network architecture

For each robotic arm, the actor network has an input size equal to the state size, i.e. 33. Then two hidden layers with 400 and 300 nodes follow and the output layer has 4 nodes, which matches the dimensionality of our action size.

The critic has a similar architecture except for the output layer, which only has one node that outputs the expected reward.

The activation functions are mostly ReLU except for the output layer, where `tanh` is applied in the actor to exhaust the range (-1,1) or no activation function in the critic to allow for any value as result. This is similar to the Pendulum project, where only the input and action sizes differ.

Please note that I also tried out the DNN from the [Bipedal project in Udacity's Deep Reinforcement Learning Repository](https://github.com/udacity/deep-reinforcement-learning/tree/master/ddpg-bipedal), where the actor has one hidden layer less and the critic one hidden layer more. The results during training were disappointing, but I decided not to further investigate why.

#### Hyperparameters
I chose the following hyperparameters:
- replay buffer size of 1e6 is 10x larger than for the Pendulum project, but for 20 agents and 1000 time steps per episodes, this allows to memorize 50 full episodes, which sounded enough to me
- learning rate for actor and critic of 1e-4. The learning rate of the critic in the Pendulum project is 10x higher, but as the course materials suggested some gradient clipping for the trainng of the critic, I decided to play it safe also for the learning rate
- the discount factor of 0.95 is slightly lower than in the Pendulum project (0.99). I think 0.95 is appropriate, because the episodes are quite long and the long term reward shouldn't influence the short term actions too much.
- the soft update parameter tau is 1e-3, which means 0.1% of the weights are transferred by each update. This is the same as in the Pendulum project
- the minibatch size is 128, also the same as in the Pendulum project
- for sharing experiences, I decided to train all agents with 10 minibatches every 20 time steps as was suggested in the course materials

Please note that the training does take some time, which means these choices were based less on rigorous experimentation but rather a combination of some experiments and gut feeling.

### Training results
If you look at the reward plot for the final configuration, you see that the reachers (each with it's own curve) learn very fast:


[image2]: https://raw.githubusercontent.com/hullmann/continuouscontrol-DDPG/master/scoregraph.png "Reward plot"

![image2]
The environment is solved if we receive an average reward of more than +30 in 100 episodes. The first score (average over all agents) over 30 is already achieved after 9 episodes. Since the requirement was "average over 100 episodes" I let the training run for 100 episodes.

A [video of the trained agents](https://github.com/hullmann/continuouscontrol-DDPG/blob/master/Twenty_reachers_trained.mp4) is  part of the repository as are the [weights of the agents](https://github.com/hullmann/continuouscontrol-DDPG/tree/master/model_weights).

# Ideas for future work

I was fascinated by the effect of parallel training and experience sharing.
I think it would be great to extend the source code such that
- the agents run and train as separated processes ("embarrassingly parallel") such that the runtime can be kept low even if we have 20 agents in parallel
- also the network weights and not only the experience replay buffer are shared among the agents and to evaluate the -- hopefully positive -- effect of this exchange